In [2]:
from selenium import webdriver  # Импортируем WebDriver для управления браузером
from selenium.webdriver.common.by import By  # Импортируем локаторы (By.XPATH, By.CSS_SELECTOR и т.д.)
from selenium.webdriver.support.ui import WebDriverWait  # Явные ожидания
from selenium.webdriver.support import expected_conditions as EC  # Готовые условия для ожиданий
import time  # Паузы между действиями (иногда полезны для динамических страниц)

In [3]:
url = 'https://www.timeshighereducation.com/world-university-rankings/latest/world-ranking'

То что внизу написано не ошибка, я просто добавила комментарии, а поом остановила выполнение кода, если его запустить и дождаться конца выполнения, все будет работать:)

Код открывает страницу в браузере, затем при необходимости закрывает баннер cookies (если находит кнопку “Allow all”), после чего ждёт, пока на странице появится таблица со строками. Дальше он пробует переключиться на вкладку “Key statistics”, чтобы таблица/данные были именно из этого раздела.

После этого начинается основной цикл сбора данных: код много раз подряд заново считывает все строки таблицы, вытаскивает текст из ячеек и пытается достать ссылку на страницу вуза из второй колонки. Каждую строку он сохраняет только один раз — проверяет уникальность по ссылке (чтобы не было дублей). Между итерациями он аккуратно прокручивает таблицу вниз, потому что на таких страницах новые строки подгружаются только при скролле.

Когда на протяжении большого числа повторов новые уникальные строки перестают появляться, код считает, что подгрузка закончилась, завершает цикл, закрывает браузер и выводит, сколько всего уникальных университетов удалось собрать.

In [4]:
driver = webdriver.Chrome()  # Запускаем Chrome под управлением Selenium (создаём объект драйвера)
driver.maximize_window()  # Разворачиваем окно браузера на весь экран
driver.get(url)  # Открываем страницу по адресу из переменной url

# Закрыть окно куки  # Комментарий-блок: дальше пробуем принять cookies, если баннер есть
try:  # Начинаем попытку (баннер может не появиться — это не ошибка)
    accept_btn = WebDriverWait(driver, 10).until(  # Ждём до 10 секунд появления кликабельной кнопки
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Allow all')]"))  # Ищем кнопку по тексту "Allow all"
    )  # Закрываем until(...) — после этого accept_btn должен быть кликабельным элементом
    accept_btn.click()  # Кликаем по кнопке обычным кликом
    time.sleep(2)  # Даём странице 2 секунды, чтобы баннер исчез и DOM/верстка стабилизировались
except:  # Если кнопки нет/не успела появиться/не кликается
    pass  # Просто игнорируем и идём дальше (для сайтов без баннера)

# Подождать таблицу  # Комментарий-блок: ждём, пока таблица прогрузится
WebDriverWait(driver, 10).until(  # Ждём до 10 секунд
    EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))  # Пока в таблице не появится хотя бы одна строка
)  # Закрываем ожидание таблицы

try:  # Пробуем переключиться на вкладку "Key statistics" (если она есть)
    key_stats_tab = WebDriverWait(driver, 10).until(  # Ждём до 10 секунд кликабельности вкладки
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Key statistics')]"))  # Ищем кнопку-вкладку по тексту
    )  # Закрываем until(...) — на выходе получаем WebElement вкладки
    key_stats_tab.click()  # Кликаем по вкладке, чтобы открыть нужный раздел
    time.sleep(2)  # Ждём 2 секунды, чтобы контент вкладки успел подгрузиться
except:  # Если вкладки нет или клик не удался
    pass  # Игнорируем и продолжаем (может быть уже открыто или вкладки нет)

# Собирать ВСЕ строки  # Комментарий-блок: дальше парсим таблицу
all_data = []  # Список, куда будем складывать данные строк таблицы
seen_hrefs = set()  # Множество для уникальности ссылок (чтобы не собирать один и тот же вуз дважды)

last_new_count = 0  # Сколько уникальных строк было собрано на прошлой итерации
no_change_count = 0  # Сколько итераций подряд не появляется новых уникальных строк
scroll_index = 0  # Индекс строки, к которой будем постепенно скроллить

for attempt in range(2000):  # Делаем до 2000 циклов чтения/скролла (защита от бесконечного цикла)
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")  # Находим все текущие строки таблицы на странице

    for row in rows:  # Проходим по каждой строке таблицы
        cells = row.find_elements(By.TAG_NAME, "td")  # Находим все ячейки td внутри конкретной строки
        if len(cells) == 0:  # Если вдруг ячеек нет (нестандартная строка)
            continue  # Пропускаем эту строку и идём дальше

        row_data = [cell.text.strip() for cell in cells]  # Забираем текст из каждой ячейки, обрезаем пробелы

        if not any(row_data):  # Если все элементы row_data пустые
            continue  # Пропускаем пустую строку

        # Извлечь ссылку на страницу вуза  # Комментарий-блок: пробуем вытащить href
        try:  # Ссылка может отсутствовать — это нормально
            link_element = cells[1].find_element(By.TAG_NAME, "a")  # ВО 2-й колонке (индекс 1) ищем ссылку <a>
            uni_url = link_element.get_attribute("href")  # Берём значение href (адрес страницы вуза)
        except:  # Если <a> нет или другая структура
            uni_url = ""  # Ставим пустую строку (значит, ссылки нет)

        # Добавлять только если ссылка уникальная  # Комментарий-блок: фильтруем дубли
        if uni_url and uni_url not in seen_hrefs:  # Если ссылка не пустая и ещё не встречалась
            seen_hrefs.add(uni_url)  # Запоминаем ссылку в множестве, чтобы не добавлять повторно
            row_data.append(uni_url)  # Добавляем ссылку в конец данных строки (как последнюю колонку)
            all_data.append(row_data)  # Сохраняем строку (с данными + ссылкой) в общий список

    current_total = len(all_data)  # Считаем, сколько всего уникальных строк уже собрали

    if current_total > last_new_count:  # Если общее количество увеличилось — появились новые строки
        last_new_count = current_total  # Обновляем “последнее” количество
        no_change_count = 0  # Сбрасываем счётчик итераций без изменений
    else:  # Если новых строк не добавилось
        no_change_count += 1  # Увеличиваем счётчик “без изменений”
        if no_change_count >= 25:  # Если 25 итераций подряд ничего нового
            break  # Выходим из цикла — скорее всего, таблица больше не подгружается

    # Скроллить постепенно  # Комментарий-блок: делаем скролл, чтобы триггерить подгрузку/виртуализацию таблицы
    if len(rows) > 10:  # Скролл имеет смысл, если строк больше 10
        target_index = min(len(rows) - 5, scroll_index)  # Берём индекс цели: не самый низ, чтобы был запас для подгрузки
        if target_index < len(rows):  # Проверяем, что индекс не вышел за границы списка
            driver.execute_script("""  # Выполняем JS-команду в браузере
                arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});
            """, rows[target_index])  # Скроллим так, чтобы выбранная строка оказалась примерно по центру
        scroll_index += 3  # Сдвигаем “курсор скролла” дальше, чтобы не стоять на месте

    time.sleep(1)  # Пауза 1 секунда, чтобы дать странице время подгрузить новые строки/данные

driver.quit()  # Закрываем браузер и завершаем сессию Selenium
print(f"Всего университетов: {len(all_data)}")  # Печатаем итоговое число собранных уникальных университетов


MaxRetryError: HTTPConnectionPool(host='localhost', port=60990): Max retries exceeded with url: /session/46745c6bf6a62e80cf6abddb318e5cae/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x121c1e520>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [5]:
import pandas as pd

df = pd.DataFrame(all_data)
df.columns = ["Rank", "Name", "No. of FTE students", "No. of students per staff", "International students", "Female:Male ratio", "URL"]
df.to_csv("statistics.csv", index=False, encoding="utf-8")

KeyboardInterrupt: 